In [145]:
import random

In [146]:
def mod_exp(base, exp, mod):
    result = 1
    base = base % mod
    while exp > 0:
        if exp % 2 == 1:
            result = (result * base) % mod
        exp = exp >> 1
        base = (base * base) % mod
    return result

In [147]:
def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

In [148]:
def generate_elgamal_keys(p, g):
    # x = random.randint(2, p - 2)
    x = 296910705850334802942252230016917197119761411529502398856920411716398776665995117253388109347257855432665575504925621092927500886757763228288976190435218181204860246792441747727687404284118803628280292427401702743706695331998886442243964393075281831948230289332261342065707450523522625876551455787909702212448
    y = mod_exp(g, x, p)
    return (p, g, y), x

In [ ]:
def encrypt_elgamal(message, public_key):
    p, g, y = public_key
    #Khóa phiên k được random mỗi lần chạy vậy nên bản mã sẽ khác nhau
    #Trong ví dụ trong file word đính kèm, khóa phiên k có giá trị
    
    #k = 231829683515004285894248732404818838505780640291715123088889449666416885328634740604061367009552293107004845199650478342025302430924465040754743126387968600172838303296241667234672358030056988574445077392877459316860987264706489823044426733783629485383174221974246710107609888722370369714382438004845162844505
    
    #Hàm tính k ngẫu nhiên
    k = random.randint(2, p - 2)
    while gcd(k, p - 1) != 1:
        k = random.randint(2, p - 2)
    #Kết thúc hàm tính k ngẫu nhiên
    print(f"k: {k}")
    s = mod_exp(y, k, p)
    c1 = mod_exp(g, k, p)
    message_int = int.from_bytes(message.encode(), 'big')
    c2 = (message_int * s) % p
    
    return c1, c2

In [150]:
def decrypt_elgamal(ciphertext, private_key, p):
    c1, c2 = ciphertext
    x = private_key
    
    s = mod_exp(c1, x, p)
    
    s_inverse = pow(s, -1, p)
    
    message_int = (c2 * s_inverse) % p
    print(f"Bản rõ: {message_int}")

    message_bytes = message_int.to_bytes((message_int.bit_length() + 7) // 8, 'big')
    return message_bytes.decode()

In [151]:
def is_prime(n, k=128):
    if n == 2 or n == 3:
        return True
    if n % 2 == 0 or n < 2:
        return False
    s, d = 0, n - 1
    while d % 2 == 0:
        s += 1
        d //= 2
    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(s - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

In [152]:
def generate_large_prime(bits=1811):
    while True:
        p = random.getrandbits(bits)
        if is_prime(p):
            return p

In [153]:
def find_primitive_root(p):
    if p == 2:
        return 1  

    phi = p - 1
    prime_factors = find_prime_factors(phi)
    
    for g in range(2, min(p, 1000)):  
        is_primitive = True
        for q in prime_factors:
            if pow(g, phi // q, p) == 1:
                is_primitive = False
                break
        if is_primitive:
            return g  
    
    return None

In [154]:
def find_prime_factors(n):
    factors = set()
    while n % 2 == 0:
        factors.add(2)
        n //= 2
    for i in range(3, int(n**0.5) + 1, 2):
        while n % i == 0:
            factors.add(i)
            n //= i
    if n > 2:
        factors.add(n)
    return factors


In [155]:
p = 344590376537974420951053201976517870797868340281010822474899522582054505771199770060111007551597043514444033262162310176556235037673574689699048504636751304765603116125059696959222855931469499847423595491860730347032761601704196496912109094192456362651535855045072467954889065252293306360209806356757850184303
# p = generate_large_prime(1024)
g = 13496366429637966183200866655908501844287542699618604731918713325753280286175497136016236614727621674635809206897089783790551319270201625116999741593674645525926986285973782042187324374542581942378861320998458518558357487645631881644112609323183361747147272964789761654258910528521592648905075324958458103809
print(f"p: {p}")
# g = find_primitive_root(p)
print(f"g: {g}")

public_key, private_key = generate_elgamal_keys(p, g)
print(f"Khóa công khai: {public_key}")
print(f"Kháo bí mật: {private_key}")

message = "Hoang Kim Chi"
ciphertext = encrypt_elgamal(message, public_key)
print(f"Bản mã: {ciphertext}")

decrypted_message = decrypt_elgamal(ciphertext, private_key, public_key[0])
print(f"Giải mã: {decrypted_message}")

p: 344590376537974420951053201976517870797868340281010822474899522582054505771199770060111007551597043514444033262162310176556235037673574689699048504636751304765603116125059696959222855931469499847423595491860730347032761601704196496912109094192456362651535855045072467954889065252293306360209806356757850184303
g: 13496366429637966183200866655908501844287542699618604731918713325753280286175497136016236614727621674635809206897089783790551319270201625116999741593674645525926986285973782042187324374542581942378861320998458518558357487645631881644112609323183361747147272964789761654258910528521592648905075324958458103809
Khóa công khai: (344590376537974420951053201976517870797868340281010822474899522582054505771199770060111007551597043514444033262162310176556235037673574689699048504636751304765603116125059696959222855931469499847423595491860730347032761601704196496912109094192456362651535855045072467954889065252293306360209806356757850184303, 13496366429637966183200866655908501844287542699